# Time Series Pipeline: Microbiome data
    For all designations/subclassifications of species in the microbiome, do the following
    time series tests.
    
    1. For each time point, find classifications where its z-score deviates from all other time points
    2. Treat each time point as the last time point, find linear trends
    3. Do changepoint analysis based on differences between population and sliding window.

In [4]:
import sys

# User Libraries
import tanner.stats.timeseries as ts
import tanner.stats.helpers as shelp
import tanner.microbiome.data as mb
import tanner.visual.timeseries as vts

# Python Libraries
import pandas as pd
import os 
import seaborn as sns

# Ipython Configuration
%pylab inline
%load_ext autoreload
%autoreload 2

# Data and analysis paths
microbiome_path = "/mounts/tscc/projects/Li-Fraumeni/data/family3/microbiome/14009b/aggregated/"
analysis_path = "/mounts/tscc/projects/Li-Fraumeni/analysis/microbiome"

Populating the interactive namespace from numpy and matplotlib


In [71]:
date = "0912015"
analysis_path = os.path.join(analysis_path, date)
if not os.path.exists(analysis_path): os.makedirs(analysis_path)
sns.set_context("talk", font_scale=1.2)
sns.set_style("whitegrid")

In [72]:
os.listdir(microbiome_path)

['tigrfam-mainrole-ann.txt',
 'tigrfam-subrole-ann.txt',
 'Eukaryota-class-abundance.txt',
 'Schork-11-samples-taxon-and-function.xlsx',
 'kog-ann.txt',
 'superkingdom-reads-count.txt',
 'Bacteria-genus-abundance.txt',
 'Bacteria-order-abundance.txt',
 'Eukaryota-family-abundance.txt',
 'cog-class-ann.txt',
 'cog-ann.txt',
 'Bacteria-phylum-abundance.txt',
 'Bacteria-class-abundance.txt',
 'kog-class-ann.txt',
 'Eukaryota-toprank-abundance.txt',
 'tigrfam-ann.txt',
 'Bacteria-toprank-abundance.txt',
 'pfam-ann.txt',
 'Eukaryota-genus-abundance.txt',
 'Bacteria-species-abundance.txt',
 'Bacteria-family-abundance.txt',
 'Eukaryota-order-abundance.txt',
 'Eukaryota-phylum-abundance.txt',
 'Eukaryota-species-abundance.txt']

###  Go through each file type and create relevant plots

In [ ]:
for fn in os.listdir(microbiome_path)[12:]:
    print fn
    if not fn.endswith('.txt'):
        continue
    folder = fn.split('.')[0]
    folder = os.path.join(analysis_path, folder)
    if not os.path.exists(folder): os.makedirs(folder)
    sample_fn = os.path.join(microbiome_path, fn)
    data_df = mb.load_aggregated(sample_fn)
    
    # Remove duplicate columns
    data_df = data_df.T.groupby(level=0).first().T
    
    # Outliers
    outliers = ts.outliers(data_df, z_threshold=3, 
                           include_sample=True)
    for sample, classification in outliers.iteritems():
        if len(classification) > 0:
            date = str(sample).split(' ')[0]
            file_prefix = os.path.join(folder, date+'_outliers')
            vts.default(data_df[classification], fn_prefix=file_prefix,
                        main_title=sample, marksample=sample)
    
    # Linear Regression:
    pvalues = ts.regress(data_df)
    file_prefix = os.path.join(folder, 'lineartrend')
    significant = pvalues[pvalues < 1e-02].index
    if len(significant) > 0:
        vts.default(data_df[significant], 
                    fn_prefix = file_prefix,
                    main_title='Unadjusted p-value < 0.01', 
                    bestfit=True)
    
    
    # Changepoints:
    file_prefix = os.path.join(folder, 'changepoints')
    changepoints = ts.changepoints(data_df, threshold=3)
    changepoints = changepoints[changepoints > 0]
    if len(changepoints) > 0:
        vts.default(data_df[changepoints.index], 
                    fn_prefix = file_prefix,
                    main_title = 'Changepoint', 
                    changepoints = changepoints.values)
   
    

Bacteria-class-abundance.txt
kog-class-ann.txt
Eukaryota-toprank-abundance.txt
tigrfam-ann.txt

In [68]:
high_threshold_path = os.path.join(analysis_path, "high_threshold")
if not os.path.exists(high_threshold_path): os.makedirs(high_threshold_path)

### Higher threshold for plotting results; .25 / num of classifications. Not quite bonferroni, but good starting point.

In [69]:
for fn in os.listdir(microbiome_path):
    print fn
    if not fn.endswith('.txt'):
        continue
    folder = fn.split('.')[0]
    folder = os.path.join(high_threshold_path, folder)
    if not os.path.exists(folder): os.makedirs(folder)
    sample_fn = os.path.join(microbiome_path, fn)
    data_df = mb.load_aggregated(sample_fn)
    
    # Remove duplicate columns
    data_df = data_df.T.groupby(level=0).first().T
    
    # Zscore cutoffs:
    pvalue = .25/len(data_df.columns)
    zscore = abs(stats.norm.ppf(pvalue))
    
    # Outliers
    outliers = ts.outliers(data_df, z_threshold=zscore, 
                           include_sample=True)
    for sample, classification in outliers.iteritems():
        if len(classification) > 0:
            date = str(sample).split(' ')[0]
            file_prefix = os.path.join(folder, date+'_outliers')
            vts.default(data_df[classification], fn_prefix=file_prefix,
                        main_title=sample, marksample=sample)
    
    # Linear Regression:
    pvalues = ts.regress(data_df)
    file_prefix = os.path.join(folder, 'lineartrend')
    significant = pvalues[pvalues < pvalue].index
    if len(significant) > 0:
        vts.default(data_df[significant], 
                    fn_prefix = file_prefix,
                    bestfit=True)
    
    
    # Changepoints:
    file_prefix = os.path.join(folder, 'changepoints')
    changepoints = ts.changepoints(data_df, threshold=zscore)
    changepoints = changepoints[changepoints > 0]
    if len(changepoints) > 0:
        vts.default(data_df[changepoints.index], 
                    fn_prefix = file_prefix,
                    main_title = 'Changepoint', 
                    changepoints = changepoints.values)
   
    

tigrfam-mainrole-ann.txt
tigrfam-subrole-ann.txt
Eukaryota-class-abundance.txt
Schork-11-samples-taxon-and-function.xlsx
kog-ann.txt
superkingdom-reads-count.txt
Bacteria-genus-abundance.txt
Bacteria-order-abundance.txt
Eukaryota-family-abundance.txt
cog-class-ann.txt
cog-ann.txt
Bacteria-phylum-abundance.txt
Bacteria-class-abundance.txt
kog-class-ann.txt
Eukaryota-toprank-abundance.txt
tigrfam-ann.txt
Bacteria-toprank-abundance.txt
pfam-ann.txt
Eukaryota-genus-abundance.txt
Bacteria-species-abundance.txt
Bacteria-family-abundance.txt
Eukaryota-order-abundance.txt
Eukaryota-phylum-abundance.txt
Eukaryota-species-abundance.txt
